In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import train_test_split
import time
import os
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Set random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
dataset = 'detailed'

# Early stopping parameters
patience = 100
max_epochs = 1

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using {device}")

using cpu


In [5]:
# Hinged Square Loss
class SquaredHingeLoss(nn.Module):
    def __init__(self, margin=1):
        super(SquaredHingeLoss, self).__init__()
        self.margin = margin

    def forward(self, predicted, y):
        low, high = y[:, 0:1], y[:, 1:2]
        loss_low = torch.relu(low - predicted + self.margin)
        loss_high = torch.relu(predicted - high + self.margin)
        loss = loss_low + loss_high
        return torch.mean(torch.square(loss))

In [6]:
class MLPModel(nn.Module):
    def __init__(self, input_size, layer_sizes):
        super(MLPModel, self).__init__()
        layers = []
        prev_size = input_size
        
        # Create hidden layers
        for size in layer_sizes:
            layers.append(nn.Linear(prev_size, size))
            layers.append(nn.ReLU())
            prev_size = size
        
        # Add output layer
        layers.append(nn.Linear(prev_size, 1))  # Output layer

        self.model = nn.Sequential(*layers)  # Combine layers into a sequential model

    def forward(self, x):
        return self.model(x)

In [7]:
# Load fold and target data
folds_df = pd.read_csv(f'../../training_data/{dataset}/folds.csv')
target_df = pd.read_csv(f'../../training_data/{dataset}/target.csv')
features_df = pd.read_csv(f'../../training_data/{dataset}/features.csv')

# Prepare CSV file for logging
report_path = f'report_{dataset}.csv'
report_header = ['dataset', 'num_layers', 'layer_size', 'test_fold', 'stop_epoch', 'train_loss', 'val_loss', 'test_loss', 'time']
if not os.path.exists(report_path):
    pd.DataFrame(columns=report_header).to_csv(report_path, index=False)

In [8]:
# Configs
configurations = {
    f'num_layers_{num_layers}_layer_size_{layer_size}_test_fold_{test_fold}': {
        'num_layers': num_layers,
        'layer_size': layer_size,
        'test_fold': test_fold
    }
    for num_layers in range(1, 4)
    for layer_size in [2, 4, 8, 16, 32, 64, 128, 256, 512]
    for test_fold in range(1, 7)
}

config_list = list(configurations.values())

In [9]:
# Make sure directories exist
os.makedirs('saved_models', exist_ok=True)
os.makedirs('predictions_all', exist_ok=True)

# Iterate over each configuration in the config_list
for config in config_list:
    num_layers, layer_size, test_fold = config['num_layers'], config['layer_size'], config['test_fold']

    # Record start time
    fold_start_time = time.time()

    # Split data into training and test sets based on the fold
    train_ids = folds_df[folds_df['fold'] != test_fold]['sequenceID']
    test_ids = folds_df[folds_df['fold'] == test_fold]['sequenceID']

    # Prepare train and test sequences as arrays
    features_train = features_df[features_df['sequenceID'].isin(train_ids)].iloc[:, 1:].to_numpy()
    target_train = target_df[target_df['sequenceID'].isin(train_ids)].iloc[:, 1:].to_numpy()
    features_test = features_df[features_df['sequenceID'].isin(test_ids)].iloc[:, 1:].to_numpy()

    # Normalize training features
    scaler = MinMaxScaler()  # Create scaler instance
    features_train = scaler.fit_transform(features_train)  # Fit on training data
    features_test = scaler.transform(features_test)  # Transform test data using the same parameters

    # Convert target data to tensors
    target_test = torch.tensor(target_df[target_df['sequenceID'].isin(test_ids)].iloc[:, 1:].to_numpy(), dtype=torch.float32)

    # Split training data into subtrain and validation sets
    X_subtrain, X_val, y_subtrain, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)
    y_subtrain = torch.tensor(y_subtrain, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)

    # Initialize the MLP model, loss function, and optimizer
    layer_sizes = [layer_size] * num_layers
    model = MLPModel(X_subtrain.shape[1], layer_sizes).to(device)
    criterion = SquaredHingeLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters())

    # Variables for early stopping
    best_val_loss, patience_counter = float('inf'), 0
    best_model_state, stop_epoch = None, 0

    # Training loop
    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        predictions = model(torch.tensor(X_subtrain, dtype=torch.float32))  # No need to convert to tensor again
        loss = criterion(predictions, y_subtrain)
        loss.backward()
        optimizer.step()

        # Evaluation phase
        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(torch.tensor(X_val, dtype=torch.float32)), y_val)
            avg_test_loss = criterion(model(torch.tensor(features_test, dtype=torch.float32)), target_test)

        avg_train_loss = loss.item()

        if epoch % 1000 == 0:
            print(f'Test fold {test_fold} \t Epoch [{epoch:3d}] \t Avg Train Loss: {avg_train_loss:.8f} \t Avg Val Loss: {val_loss.item():.8f} \t Avg Test Loss: {avg_test_loss.item():.8f}')

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss, best_model_state = val_loss.item(), model.state_dict()
            patience_counter = 0
            stop_epoch = epoch
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at Epoch [{epoch}]")
            break

    # Restore best model state for final evaluation
    if best_model_state:
        model.load_state_dict(best_model_state)
    
    # Save model parameters
    torch.save(model.state_dict(), f'saved_models/{dataset}_{num_layers}layers_{layer_size}neurons_fold{test_fold}.pth')

    # Record end time and calculate elapsed time
    elapsed_time = time.time() - fold_start_time

    # Log the results
    report_entry = {
        'dataset': dataset,
        'num_layers': num_layers,
        'layer_size': layer_size,
        'test_fold': test_fold,
        'stop_epoch': stop_epoch,
        'train_loss': avg_train_loss,
        'val_loss': best_val_loss,
        'test_loss': avg_test_loss.item(),
        'time': elapsed_time
    }
    pd.DataFrame([report_entry]).to_csv(report_path, mode='a', header=False, index=False)

    # Predict on the test set and save to CSV
    model.eval()
    pred_lldas = model(torch.tensor(features_test, dtype=torch.float32)).detach().numpy().ravel()  # No need to convert to tensor again
    lldas_df = pd.DataFrame({'sequenceID': features_df[features_df['sequenceID'].isin(test_ids)]['sequenceID'], 'llda': pred_lldas})
    lldas_df.to_csv(f'predictions_all/{dataset}.{num_layers}layers_{layer_size}neurons_{test_fold}.csv', index=False)

Test fold 1 	 Epoch [  0] 	 Avg Train Loss: 0.48779464 	 Avg Val Loss: 0.50346583 	 Avg Test Loss: 0.46622780
Test fold 2 	 Epoch [  0] 	 Avg Train Loss: 0.47449434 	 Avg Val Loss: 0.35255274 	 Avg Test Loss: 0.54014379
Test fold 3 	 Epoch [  0] 	 Avg Train Loss: 0.47042167 	 Avg Val Loss: 0.57191396 	 Avg Test Loss: 0.43236721
Test fold 4 	 Epoch [  0] 	 Avg Train Loss: 0.44777322 	 Avg Val Loss: 0.46341816 	 Avg Test Loss: 0.42979530
Test fold 5 	 Epoch [  0] 	 Avg Train Loss: 0.47739053 	 Avg Val Loss: 0.43064213 	 Avg Test Loss: 0.49505320
Test fold 6 	 Epoch [  0] 	 Avg Train Loss: 0.68149590 	 Avg Val Loss: 0.51733661 	 Avg Test Loss: 0.61558694
Test fold 1 	 Epoch [  0] 	 Avg Train Loss: 0.46600819 	 Avg Val Loss: 0.48861468 	 Avg Test Loss: 0.46075675
Test fold 2 	 Epoch [  0] 	 Avg Train Loss: 0.46485791 	 Avg Val Loss: 0.35067022 	 Avg Test Loss: 0.52883333
Test fold 3 	 Epoch [  0] 	 Avg Train Loss: 0.43610173 	 Avg Val Loss: 0.52639842 	 Avg Test Loss: 0.38731730
Test fold 